In [1]:
import sys
import os
sys.path.append('../src-py/')
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
from ca_utils import *

2022-05-19 18:28:39,883 loading file ../../../data-ceph/arguana/arg-generation/claim-target-tagger/model/final-model.pt
2022-05-19 18:29:10,250 SequenceTagger predicts: Dictionary with 5 tags: O, S-CT, B-CT, E-CT, I-CT


In [3]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

pd.set_option('display.max_colwidth', None)

In [4]:
import torch

In [5]:
torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

device(type='cuda')

### Generate conclusions for the posts:

In [10]:
def generate_post_conclusions(df):
    
    gen_kwargs = {
        "do_sample": True, 
        "max_length":20,
        "top_p":0.95, 
        "num_beams":1
    }

    post_dict = list(pd.Series(df.post.values, index=df.post_id).to_dict().items())
    post_ids, posts = zip(*post_dict)
    pred_conclusions = generate_conclusion(posts, gen_kwargs, batch_size=8)
    post_pred_conclusions = {x[0]: x[1] for x in zip(post_ids, pred_conclusions)}
    df['bart_conclusion'] = df.post_id.apply(lambda x: post_pred_conclusions[x])
    
    return df

#### For a sample of the test-dataset:

In [8]:
test_df = pd.read_pickle('../../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/sample_test_conclusion_all.pkl')
test_df = generate_post_conclusions(test_df)
test_df.to_pickle('../../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/sample_test_conclusion_all_preprocessed.pkl')

#### For a sample of the valid-dataset:

In [11]:
valid_df = pd.read_pickle('../../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/sample_valid_conclusion_all.pkl')
test_df = generate_post_conclusions(valid_df)
valid_df.to_pickle('../../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/sample_valid_conclusion_all_preprocessed.pkl')

  0%|          | 0/2 [00:00<?, ?ba/s]

/usr/local/lib/python3.6/dist-packages/transformers/generation_utils.py:2142: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


#### For the whole test-dataset:

In [16]:
test_df = pd.read_pickle('../../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/test_conclusion_all.pkl')
test_df = test_df.drop_duplicates('post_id')
test_df = generate_post_conclusions(test_df)
test_df.to_pickle('../../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/test_conclusion_all_preprocessed.pkl')

  0%|          | 0/9 [00:00<?, ?ba/s]

In [17]:
test_df[['title', 'bart_conclusion']].head()

,title,bart_conclusion
410850,people should come with instructions,i think people should be required by law to use a cheat sheet if they meet someone they
410858,People should not be heavily criticized for things they put on social media in the distant past,i think the internet should stop being as harsh on people for things they put on social
410902,We shouldn't focus on slowing climate change,joint statement:: there are other environmental issues that are a greater problem for
410910,The Australian PM was right to tell students to stop activism around global warming,I believe that activism is a terrible way to combat climate change
410916,Feeding cats or dogs a diet with meat is indefensible.,if a cat or dog eats her life then it's a animal killer and they should be


In [18]:
len(test_df)

8535